In [3]:
df

,dataset,num_institution,num_institution_user,num_anchor_data,dim_intermediate,dim_integrate,h_model,f_seed,metrics,F_type,...,integ_metrics,losses_even_ind,losses_odd_ind,losses_ind,losses_even,losses_odd,losses_mean,評価値,error,gamma_ratio_krr


In [6]:
import pandas as pd

# CSVファイルの読み込み
file_path = "result_grid_mice copy.csv"
data = pd.read_csv(file_path)

# 必要な列を固定する
fixed_columns = [
    "dataset", "h_model", "F_type", "G_type", "gamma_type", 
    "gamma_ratio", "num_anchor_data", "dim_intermediate", 
    "num_institution_user", "loop_num"
]

# 固定列ごとにグループ化し、score_meanが最大となる行を取得
grouped = data.groupby(fixed_columns, as_index=False).apply(
    lambda group: group.loc[group['score_mean'].idxmax()]
)

# gamma_ratio_krrとnl_lambdaの組み合わせをカウント
combination_counts = grouped.groupby(["gamma_ratio_krr", "nl_lambda"]).size().reset_index(name="count")

# カウントの降順でソート
ranked_combinations = combination_counts.sort_values(by="count", ascending=False).reset_index(drop=True)

# 結果を表示
print(ranked_combinations)

# 必要に応じてCSVに保存
output_file = "ranked_combinations.csv"
ranked_combinations.to_csv(output_file, index=False)
print(f"ランキング結果を {output_file} に保存しました。")

   gamma_ratio_krr  nl_lambda  count
0              0.2    0.00001     81
1              0.2    0.10000     60
2              1.0    0.00001      8
3              1.0    0.10000      6
4              5.0    0.10000      3
5              5.0    0.00001      2
ランキング結果を ranked_combinations.csv に保存しました。


C:\Users\sueya\AppData\Local\Temp\ipykernel_23276\637859921.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = data.groupby(fixed_columns, as_index=False).apply(


In [9]:
import pandas as pd

# CSVファイルの読み込み
file_path = "result_grid_mice copy.csv"
data = pd.read_csv(file_path)
# gamma_ratio=10 を除外
data = data[data["gamma_ratio"] != 10]
# 必要な列を固定する
fixed_columns = [
    "dataset", "h_model", "F_type", "G_type", "gamma_type", 
    "gamma_ratio", "num_anchor_data", "dim_intermediate", 
    "num_institution_user", "loop_num"
]

# 固定列ごとにグループ化し、score_meanが最大となる行を取得
grouped = data.groupby(fixed_columns, as_index=False).apply(
    lambda group: group.loc[group['score_mean'].idxmax()]
)

# gamma_ratio_krrとnl_lambdaの組み合わせをカウントし、score_meanの平均値も計算
combination_stats = grouped.groupby(["gamma_ratio_krr", "nl_lambda"]).agg(
    count=("score_mean", "size"),  # 組み合わせのカウント
    mean_score=("score_mean", "mean")  # score_mean の平均値
).reset_index()

# カウントの降順でソート
ranked_combinations = combination_stats.sort_values(by="count", ascending=False).reset_index(drop=True)

# 結果を表示
print(ranked_combinations)

# 必要に応じてCSVに保存
output_file = "ranked_combinations_with_mean.csv"
ranked_combinations.to_csv(output_file, index=False)
print(f"ランキング結果を {output_file} に保存しました。")

   gamma_ratio_krr  nl_lambda  count  mean_score
0              0.2    0.00001     73    0.801832
1              0.2    0.10000     36    0.763895
2              1.0    0.00001      8    0.686580
3              1.0    0.10000      6    0.746543
4              5.0    0.10000      3    0.592554
5              5.0    0.00001      2    0.577894
ランキング結果を ranked_combinations_with_mean.csv に保存しました。


C:\Users\sueya\AppData\Local\Temp\ipykernel_23276\668075369.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = data.groupby(fixed_columns, as_index=False).apply(


In [31]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from pathlib import Path

# 入力CSV
csv_path = r"result_grid_mice.csv"

# ロード（エンコーディング自動試行）
def load_csv(path: str) -> pd.DataFrame:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"CSV not found: {p.resolve()}")
    for enc in ("utf-8-sig", "cp932", "utf-8"):
        try:
            return pd.read_csv(p, encoding=enc, low_memory=False)
        except Exception:
            pass
    return pd.read_csv(p, low_memory=False)

df = load_csv(csv_path)

# 必須列チェック
required_cols = [
    "dataset","h_model","F_type","gamma_type","gamma_ratio",
    "num_anchor_data","dim_intermediate","num_institution_user",
    "G_type","gamma_ratio_krr","nl_lambda",
    "score_mean","odd_mean","even_mean","odd_ind_mean","even_ind_mean",
]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"以下の列がCSVに存在しません: {missing}")

# 数値化（念のため）
num_cols = ["gamma_ratio","gamma_ratio_krr","nl_lambda",
            "score_mean","odd_mean","even_mean","odd_ind_mean","even_ind_mean"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# 識別用: "G_type+gamma_ratio_krr+nl_lambda"
def fmt_num(x):
    if pd.isna(x):
        return "NA"
    x = float(x)
    return str(int(x)) if x.is_integer() else f"{x:g}"

df["nonlinear_config_id"] = (
    df["G_type"].astype(str) + "+" +
    df["gamma_ratio_krr"].map(fmt_num) + "+" +
    df["nl_lambda"].map(fmt_num)
)

# 条件1: 同一(細)グループで baseline(=Imakura/GEP/ODC) の全てより score_mean が高い nonlinear
group_cols_full = [
    "dataset","h_model","F_type","gamma_type","gamma_ratio",
    "num_anchor_data","dim_intermediate","num_institution_user",
]
baseline_set = {"Imakura","GEP","ODC"}

def select_nonlinear_beating_baseline(dfg: pd.DataFrame) -> pd.DataFrame:
    base_mask = dfg["G_type"].isin(baseline_set)
    if not base_mask.any():
        return dfg.iloc[0:0]
    baseline_max = dfg.loc[base_mask, "score_mean"].max()
    pick = (dfg["G_type"] == "nonlinear") & (dfg["score_mean"] > baseline_max)
    return dfg.loc[pick].copy()

filtered1 = (
    df.groupby(group_cols_full, dropna=False, as_index=False, group_keys=True)
      .apply(select_nonlinear_beating_baseline)
      .reset_index(drop=True)
)

# 条件2: odd_mean > odd_ind_mean かつ even_mean > even_ind_mean
filtered2 = filtered1[
    (filtered1["odd_mean"]  > filtered1["odd_ind_mean"]) &
    (filtered1["even_mean"] > filtered1["even_ind_mean"])
].copy()

# 条件3: 同一(粗)グループの individual 1件と比較し、score/odd/even すべて上回る
group_cols_coarse = ["dataset","h_model","F_type","gamma_type","num_institution_user"]

# individual 参照（グループ毎に1行想定。複数あっても最初を採用）
ind_ref = (
    df.loc[df["G_type"] == "individual", group_cols_coarse + ["score_mean","odd_mean","even_mean"]]
      .drop_duplicates(subset=group_cols_coarse, keep="first")
      .rename(columns={
          "score_mean": "ind_score_mean",
          "odd_mean": "ind_odd_mean",
          "even_mean": "ind_even_mean",
      })
)

# マージして比較（nonlinear等の候補のみ、individual自身は除外）
cand = (
    filtered2.loc[filtered2["G_type"] != "individual"]
             .merge(ind_ref, on=group_cols_coarse, how="left", validate="m:1")
)

filtered3 = cand[
    cand["ind_score_mean"].notna() &
    cand["ind_odd_mean"].notna() &
    cand["ind_even_mean"].notna() &
    (cand["score_mean"] > cand["ind_score_mean"]) &
    (cand["odd_mean"]   > cand["ind_odd_mean"]) &
    (cand["even_mean"]  > cand["ind_even_mean"])
].copy()

# 保存（見やすいようソート）
out1 = "filtered_nonlin_vs_baselines.csv"
out2 = "filtered_nonlin_vs_baselines_odd_even_vs_ind_cols.csv"
out3 = "filtered_nonlin_vs_individual_coarsegroup.csv"

filtered1.sort_values(group_cols_full + ["nonlinear_config_id","score_mean"], inplace=True, ignore_index=True)
filtered2.sort_values(group_cols_full + ["nonlinear_config_id","score_mean"], inplace=True, ignore_index=True)
filtered3.sort_values(group_cols_coarse + ["gamma_ratio","num_anchor_data","dim_intermediate","nonlinear_config_id","score_mean"], inplace=True, ignore_index=True)

filtered1.to_csv(out1, index=False)
filtered2.to_csv(out2, index=False)
filtered3.to_csv(out3, index=False)

print(f"[INFO] 条件1クリア数: {len(filtered1)} -> {out1}")
print(f"[INFO] 条件2クリア数: {len(filtered2)} -> {out2}")
print(f"[INFO] 条件3クリア数: {len(filtered3)} -> {out3}")

# ---------------- チェックコード ----------------

# 条件1チェック: baseline_max を上回る nonlinear のみ含むか
def build_baseline_max_map(dall: pd.DataFrame):
    m = {}
    for keys, dfg in dall.groupby(group_cols_full, dropna=False):
        base_mask = dfg["G_type"].isin(baseline_set)
        if base_mask.any():
            m[keys] = dfg.loc[base_mask, "score_mean"].max()
    return m

baseline_max_map = build_baseline_max_map(df)

def assert_condition1_ok(dsub: pd.DataFrame):
    bad = []
    for i, row in dsub.iterrows():
        keys = tuple(row[c] for c in group_cols_full)
        base_max = baseline_max_map.get(keys, np.nan)
        if pd.isna(base_max) or not (row["G_type"] == "nonlinear" and row["score_mean"] > base_max):
            bad.append((i, base_max, row["score_mean"]))
    if bad:
        raise AssertionError(f"条件1違反: {len(bad)}件 例: {bad[:5]}")
    print("[CHECK] 条件1 OK")

# 条件2チェック
def assert_condition2_ok(dsub: pd.DataFrame):
    bad = dsub[~((dsub["odd_mean"] > dsub["odd_ind_mean"]) & (dsub["even_mean"] > dsub["even_ind_mean"])) ]
    if len(bad) > 0:
        raise AssertionError(f"条件2違反: {len(bad)}件")
    print("[CHECK] 条件2 OK")

# 条件3チェック: 粗キー一致の individual 値を取り直して比較
def assert_condition3_ok(dall: pd.DataFrame, dsub: pd.DataFrame):
    # individual を辞書化
    key = lambda r: tuple(r[c] for c in group_cols_coarse)
    ind_map = {}
    for _, r in dall[dall["G_type"] == "individual"].iterrows():
        k = key(r)
        # 複数あっても最初の1件を採用
        if k not in ind_map:
            ind_map[k] = (r["score_mean"], r["odd_mean"], r["even_mean"])
    bad = []
    for i, r in dsub.iterrows():
        k = key(r)
        ref = ind_map.get(k, None)
        if ref is None:
            bad.append((i, "no individual"))
            continue
        sc, od, ev = ref
        if not (r["score_mean"] > sc and r["odd_mean"] > od and r["even_mean"] > ev):
            bad.append((i, (sc, od, ev), (r["score_mean"], r["odd_mean"], r["even_mean"])))
    if bad:
        raise AssertionError(f"条件3違反: {len(bad)}件 例: {bad[:5]}")
    print("[CHECK] 条件3 OK")

# 実行
assert_condition1_ok(filtered1)
assert_condition2_ok(filtered2)
assert_condition3_ok(df, filtered3)

# 参考表示
def peek(df_show: pd.DataFrame, n=10, cols=None, title=""):
    print(f"\n===== {title} (top {n}) =====")
    print(df_show.loc[:, cols].head(n) if cols else df_show.head(n))

show_cols1 = group_cols_full + ["G_type","nonlinear_config_id","score_mean","odd_mean","even_mean","odd_ind_mean","even_ind_mean"]
peek(filtered1, 10, show_cols1, "条件1 クリア例")
peek(filtered2, 10, show_cols1, "条件2 クリア例")
show_cols3 = group_cols_coarse + ["gamma_ratio","num_anchor_data","dim_intermediate","G_type","nonlinear_config_id","score_mean","odd_mean","even_mean"]
peek(filtered3, 10, show_cols3, "条件3 クリア例")

[INFO] 条件1クリア数: 86 -> filtered_nonlin_vs_baselines.csv
[INFO] 条件2クリア数: 29 -> filtered_nonlin_vs_baselines_odd_even_vs_ind_cols.csv
[INFO] 条件3クリア数: 0 -> filtered_nonlin_vs_individual_coarsegroup.csv
[CHECK] 条件1 OK
[CHECK] 条件2 OK
[CHECK] 条件3 OK

===== 条件1 クリア例 (top 10) =====
  dataset h_model                F_type gamma_type  gamma_ratio  \
0    mice     mlp  kernel_pca_svd_mixed   X_tuning          0.1   
1    mice     mlp  kernel_pca_svd_mixed   X_tuning          0.1   
2    mice     mlp  kernel_pca_svd_mixed   X_tuning          0.3   
3    mice     mlp  kernel_pca_svd_mixed   X_tuning          0.3   
4    mice     mlp  kernel_pca_svd_mixed   X_tuning          0.3   
5    mice     mlp  kernel_pca_svd_mixed   X_tuning          0.3   
6    mice     mlp  kernel_pca_svd_mixed   X_tuning          1.0   
7    mice     mlp  kernel_pca_svd_mixed   X_tuning          1.0   
8    mice     mlp  kernel_pca_svd_mixed   X_tuning          1.0   
9    mice     mlp  kernel_pca_svd_mixed   X_tuning      

C:\Users\sueya\AppData\Local\Temp\ipykernel_23276\1702861045.py:70: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(select_nonlinear_beating_baseline)


In [32]:
# 追加（filtered1/filtered2 を壊さず、対応 individual の score_mean を列追加）
group_cols_coarse = ["dataset","h_model","F_type","gamma_type","num_institution_user"]

ind_ref = (
    df.loc[df["G_type"] == "individual", group_cols_coarse + ["score_mean"]]
      .drop_duplicates(subset=group_cols_coarse, keep="first")
      .rename(columns={"score_mean": "ind_score"})
)
filtered1 = filtered1.merge(ind_ref, on=group_cols_coarse, how="left", validate="m:1")
filtered2 = filtered2.merge(ind_ref, on=group_cols_coarse, how="left", validate="m:1")

# 追加（filtered1/filtered2 を壊さず、同じキーの GEP/ODC/Imakura の score_mean を列追加）
keys = [
    "dataset","h_model","F_type","gamma_type","gamma_ratio",
    "num_anchor_data","dim_intermediate","num_institution_user",
]

base_ref = (
    df.loc[df["G_type"].isin(["GEP","ODC","Imakura"]), keys + ["G_type","score_mean"]]
      .drop_duplicates(subset=keys + ["G_type"], keep="first")
)

wide_ref = (
    base_ref.pivot(index=keys, columns="G_type", values="score_mean")
            .reset_index()
)
wide_ref.columns.name = None  # 列名階層を解除（列名は "GEP","ODC","Imakura" になる）

filtered1 = filtered1.merge(wide_ref, on=keys, how="left", validate="m:1")
filtered2 = filtered2.merge(wide_ref, on=keys, how="left", validate="m:1")
filtered1.to_csv(out1, index=False)
filtered2.to_csv(out2, index=False)

In [20]:
filtered1[:86]

,dataset,h_model,F_type,G_type,gamma_type,gamma_ratio,gamma_ratio_krr,num_anchor_data,nl_lambda,dim_intermediate,...,score_mean,score_stdev,even_ind_mean,odd_ind_mean,ind_mean,mean_mean,even_mean,odd_mean,integ_metrics_mean,nonlinear_config_id
0,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,0.1,0.2,100,0.00001,20,...,0.960014,0.016033,0.9022,0.7373,0.8363,0.9718,0.9726,0.9708,28.0,nonlinear+0.2+1e-05
1,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,0.1,0.2,1000,0.10000,10,...,0.919604,0.007566,0.5874,0.5866,0.5870,0.9181,0.8924,0.9437,9.9,nonlinear+0.2+0.1
2,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,0.3,0.2,1000,0.10000,5,...,0.822064,0.012232,0.6006,0.6544,0.6275,0.8233,0.8167,0.8299,8.0,nonlinear+0.2+0.1
3,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,0.3,0.2,1000,0.00001,5,...,0.830474,0.010495,0.6006,0.6544,0.6275,0.8291,0.8510,0.8072,29.9,nonlinear+0.2+1e-05
4,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,0.3,0.2,1000,0.00001,10,...,0.879773,0.009854,0.7263,0.7823,0.7487,0.8744,0.8858,0.8572,46.2,nonlinear+0.2+1e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,10.0,0.2,1000,0.00001,20,...,0.630005,0.032670,0.6152,0.7373,0.6641,0.6601,0.5305,0.8545,72.1,nonlinear+0.2+1e-05
82,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,10.0,0.2,1000,0.10000,20,...,0.625225,0.031234,0.6121,0.9230,0.7158,0.5896,0.4462,0.8765,16.9,nonlinear+0.2+0.1
83,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,10.0,0.2,1000,0.00001,20,...,0.655848,0.003188,0.6121,0.9230,0.7158,0.6595,0.5360,0.9066,91.1,nonlinear+0.2+1e-05
84,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,10.0,0.2,1000,0.10000,20,...,0.740252,0.007774,0.5803,0.9645,0.7724,0.7330,0.5125,0.9536,26.3,nonlinear+0.2+0.1


In [17]:
filtered2

,dataset,h_model,F_type,G_type,gamma_type,gamma_ratio,gamma_ratio_krr,num_anchor_data,nl_lambda,dim_intermediate,...,score_mean,score_stdev,even_ind_mean,odd_ind_mean,ind_mean,mean_mean,even_mean,odd_mean,integ_metrics_mean,nonlinear_config_id
0,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,0.1,0.2,100,0.00001,20,...,0.960014,0.016033,0.9022,0.7373,0.8363,0.9718,0.9726,0.9708,28.0,nonlinear+0.2+1e-05
1,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,0.1,0.2,1000,0.10000,10,...,0.919604,0.007566,0.5874,0.5866,0.5870,0.9181,0.8924,0.9437,9.9,nonlinear+0.2+0.1
2,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,0.3,0.2,1000,0.10000,5,...,0.822064,0.012232,0.6006,0.6544,0.6275,0.8233,0.8167,0.8299,8.0,nonlinear+0.2+0.1
3,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,0.3,0.2,1000,0.00001,5,...,0.830474,0.010495,0.6006,0.6544,0.6275,0.8291,0.8510,0.8072,29.9,nonlinear+0.2+1e-05
4,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,0.3,0.2,1000,0.00001,10,...,0.879773,0.009854,0.7263,0.7823,0.7487,0.8744,0.8858,0.8572,46.2,nonlinear+0.2+1e-05
5,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,0.3,0.2,1000,0.00001,20,...,0.943930,0.019209,0.8799,0.9230,0.8943,0.9508,0.9583,0.9359,74.0,nonlinear+0.2+1e-05
6,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,1.0,0.2,100,0.00001,20,...,0.874220,0.012647,0.8347,0.7373,0.7958,0.8787,0.8897,0.8622,27.6,nonlinear+0.2+1e-05
7,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,1.0,0.2,1000,0.00001,20,...,0.893448,0.017524,0.7183,0.6900,0.7042,0.8978,0.8817,0.9138,83.7,nonlinear+0.2+1e-05
8,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,1.0,0.2,1000,0.00001,20,...,0.887948,0.021432,0.8347,0.7373,0.7958,0.8809,0.8950,0.8597,84.8,nonlinear+0.2+1e-05
9,mice,mlp,kernel_pca_svd_mixed,nonlinear,X_tuning,3.0,0.2,100,0.00001,5,...,0.694187,0.022518,0.5907,0.6544,0.6225,0.6839,0.6546,0.7132,10.6,nonlinear+0.2+1e-05


In [18]:
filtered3

,dataset,h_model,F_type,G_type,gamma_type,gamma_ratio,gamma_ratio_krr,num_anchor_data,nl_lambda,dim_intermediate,...,odd_ind_mean,ind_mean,mean_mean,even_mean,odd_mean,integ_metrics_mean,nonlinear_config_id,ind_score_mean,ind_odd_mean,ind_even_mean
